In [59]:
#Import modules
import numpy as np
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import Dataset
import pandas as pd
from surprise import Reader
from surprise.model_selection.split import PredefinedKFold
from surprise import accuracy

In [57]:
#Load Data
folds_files = [('train.txt','test.txt')]
reader = Reader(line_format='user item rating', sep='\t', skip_lines=None, rating_scale=(1, 5))
data = Dataset.load_from_folds(folds_files,reader=reader)
pkf = PredefinedKFold()
trainset, testset = next(pkf.split(data))

In [60]:
#Set params
k=20
epochs = 100
algo = SVD(n_factors=k,n_epochs=epochs)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions,verbose=True)

RMSE: 0.9848


0.9848035463818261